# Overview

In this section, we examine job postings across Business Analytics, Data Science, and Machine Learning roles to identify key patterns before modeling. We explore salary, experience, remote work trends, and skill demand to guide the regression and classification stages.

We focused on:
- Job distribution across industries and roles  
- Salary variation by experience and remote type  
- Skill demand frequency and specialization  
- Emerging data science and analytics trends


# Load and Prepare Data

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

# Efficient load with selective columns
usecols = [
    "Average_Salary", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE",
    "REMOTE_GROUP", "ROLE_GROUP", "STATE", "TOP_SKILLS"
]
df = pd.read_csv("data/lightcast_cleaned.csv", usecols=usecols, low_memory=False)

# Downsample for memory safety on EC2
if len(df) > 8000:
    df = df.sample(8000, random_state=42)

In [ ]:
fig = px.histogram(
    df, x="Average_Salary", nbins=40,
    color_discrete_sequence=["#187145"],
    title="Distribution of Average Salaries"
)
fig.update_layout(
    xaxis_title="Average Salary ($)",
    yaxis_title="Number of Job Postings",
    template="plotly_white"
)
fig

### Salary Distribution and Outliers
Salaries are right-skewed — most roles pay under $150K, but select senior and ML-focused positions exceed $200K, highlighting outlier opportunities for experienced professionals.


In [ ]:
fig = px.scatter(
    df,
    x="MIN_YEARS_EXPERIENCE",
    y="Average_Salary",
    color="REMOTE_GROUP",
    trendline="ols",
    title="Salary vs. Minimum Experience by Remote Type",
    height=500
)
fig.update_layout(template="plotly_white")
fig

### Salary vs Experience
Salary shows a positive correlation with experience, particularly for hybrid and remote roles — suggesting advanced or flexible positions are often compensated higher.


In [ ]:
fig = px.box(
    df,
    x="ROLE_GROUP",
    y="Average_Salary",
    color="ROLE_GROUP",
    color_discrete_sequence=["#187145", "#45A274", "#79C99E"],
    title="Salary Comparison Across Role Categories"
)
fig.update_layout(template="plotly_white")
fig

### Salary by Role Category (BA / DS / ML)
Machine Learning roles exhibit the highest median salaries, followed by Data Science and Business Analytics. This validates using ROLE_GROUP as a key feature in regression modeling.


In [ ]:
if df["STATE"].notna().sum() > 0:
    fig = px.choropleth(
        df,
        locations="STATE",
        locationmode="USA-states",
        color="Average_Salary",
        scope="usa",
        color_continuous_scale="Greens",
        title="Average Salary by U.S. State"
    )
    fig
else:
    print("State data not available — skipping map.")

### Geographic and Remote Trends
Coastal states (CA, MA, NY) show higher average salaries — consistent with tech-industry concentration. These differences will be important for geographic feature engineering.


In [ ]:
fig = px.box(
    df,
    x="REMOTE_GROUP",
    y="Average_Salary",
    color="REMOTE_GROUP",
    color_discrete_sequence=["#45A274", "#79C99E", "#A7D9C9"],
    title="Salary Distribution by Remote Work Type"
)
fig.update_layout(template="plotly_white")
fig

### Remote Work vs Salary Trends
Remote roles tend to offer higher median salaries compared to onsite jobs, supporting modern hybrid compensation trends in data-driven fields.


In [ ]:
skills_flat = [
    s.strip()
    for sublist in df['TOP_SKILLS'].dropna().str.split(',')
    for s in sublist if s.strip()
]
skill_counts = pd.DataFrame(
    Counter(skills_flat).most_common(15),
    columns=['Skill', 'Count']
)

fig = px.bar(
    skill_counts,
    x='Skill',
    y='Count',
    title="Most Frequent Skills in Job Postings",
    color_discrete_sequence=["#187145"]
)
fig.update_layout(template="plotly_white")
fig

### Top Skills Frequency
Python, SQL, and Machine Learning dominate job descriptions, indicating their critical role in employability and forming the foundation for the modeling phase.


In [ ]:
corr_cols = ["Average_Salary", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE"]
corr = df[corr_cols].corr().round(2)

fig = ff.create_annotated_heatmap(
    z=corr.values,
    x=corr.columns.tolist(),
    y=corr.columns.tolist(),
    colorscale='greens',
    showscale=True
)
fig.update_layout(title="Feature Correlation Matrix", template="plotly_white", height=450)
fig

### Correlation Heatmap
Salary correlates moderately with both minimum and maximum experience, confirming their inclusion in regression modeling.
We’ll rely on additional categorical encodings (e.g., REMOTE_GROUP, ROLE_GROUP) for predictive depth.